In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
from matplotlib import pyplot

import sys
import os

#sklearn stuff
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error

#important stuff!!!
import pickle

# Save and Read Data through pickles

__If you dont have pickle files, You are can run the following two code lines only once. It will read your data in the normal way,and save it in as pickles!. (much faster that way!)__

Step 1: Reading in a classic way (takes time) (you should run this only once since later you gonna save it in the pickle format)

In [2]:
#start = time.time()

#df_train=pd.read_csv('train.csv')
#df_building_metadata=pd.read_csv('building_metadata.csv')
#df_test=pd.read_csv('test.csv')
#df_weather_test=pd.read_csv('weather_test.csv')
#df_weather_train=pd.read_csv('weather_train.csv')
sample_submission=pd.read_csv('sample_submission.csv')
#print('It took', (time.time()-start)/60, 'minutes.')

Step 2: Save it as a pickle (you also run this only once!)

In [3]:
#pickle.dump(df_train, open("train.p", "wb"))
#pickle.dump(df_building_metadata, open("building_metadata.p", "wb"))
#pickle.dump(df_test, open("test.p", "wb"))
#pickle.dump(df_weather_test, open("weather_test.p", "wb"))
#pickle.dump(df_weather_train, open("weather_train.p", "wb"))


__You should use this code for future runs (after you run the last two line only once.)__

In [4]:
start = time.time()
#df_train=pickle.load(open("train.p", "rb"))
#df_building_metadata=pickle.load(open("building_metadata.p", "rb"))
df_test=pickle.load(open("test.p", "rb"))
df_weather_test=pickle.load(open("weather_test.p", "rb"))
#df_weather_train=pickle.load(open("weather_train.p", "rb"))
df_metadata=pickle.load(open("building_metadata.p", "rb"))

print('It took', (time.time()-start)/60, 'minutes.')

It took 0.14775545597076417 minutes.


__For this notebook I use Kristofs murged file for the training dataset__

Again you run the following line only once!

In [5]:
#df_merged=pd.read_csv('mergedData.csv')
#pickle.dump(df_merged, open("mergedData.p", "wb"))

And for future runs you use this!

In [6]:
df=pickle.load(open("mergedData.p", "rb"))

# Time-stamp transformation

Why? The timestamp that they use is as a string object.
datetime object is more usefull and easier/faster to use through matplotlib and pandas functions.

In [7]:
          # FOR ALL DATA
#df_train['timestamp_2'] = pd.to_datetime(df_train['timestamp'])
df_test['timestamp_2'] = pd.to_datetime(df_test['timestamp'])
#df_weather_test['timestamp_2'] = pd.to_datetime(df_weather_test['timestamp'])
#df_weather_train['timestamp_2'] = pd.to_datetime(df_weather_train['timestamp'])

In [8]:
# FOR KRISTOFS MURGED DATA
df['timestamp_2'] = pd.to_datetime(df['timestamp'])

# Data Preparation

In [9]:
df_train=df[df['set']=='train']
del df

__Metadata merging in test:__

In [10]:
df_test = df_test.merge(df_metadata, on='building_id', how='left')

#df_metadata_for_primary_use=df_metadata_new[df_metadata_new.columns.difference(['site_id','primary_use','square_feet','floor_count','year_built'])]
#df_train = df_train.merge(df_metadata_for_primary_use, on='building_id', how='left')
#df_test = df_test.merge(df_metadata_for_primary_use, on='building_id', how='left')

#### Get dummy variables for "primary_use" 

Primary use is a categorical feature so we should make it nummerical

__Option 1:__ Get dummies that create columns based on the different types of primarly use with (0 and 1) values

In [8]:
#df_metadata.head(1)

In [9]:
#df_primary_use = pd.get_dummies(df_metadata['primary_use'])

In [10]:
#df_metadata_new = pd.concat([df_metadata, df_primary_use], axis=1)
#df_metadata_new.head(6)

The columns are a bit sparse which is not a really good thing...

__Option 2:__ Get dummy variables (0,1,2,3) in the same column instead of (Educational, Entertaining, etc. types of primary use):

In [11]:
#Converting catergorical primarly use to nummeric
df_test['primary_use'] = df_test['primary_use'].astype('category')
df_train['primary_use'] = df_train['primary_use'].astype('category')

cat_columns = df_test.select_dtypes(['category']).columns
df_test[cat_columns] = df_test[cat_columns].apply(lambda x: x.cat.codes)

cat_column = df_train.select_dtypes(['category']).columns
df_train[cat_column] = df_train[cat_column].apply(lambda x: x.cat.codes)

In [12]:
df_test.head(1)

,row_id,building_id,meter,timestamp,timestamp_2,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,2017-01-01 00:00:00,2017-01-01,0,0,7432,2008.0,NaN


In [13]:
df_train.columns

Index(['Unnamed: 0', 'site_id', 'timestamp', 'building_id', 'meter',
       'meter_reading', 'primary_use', 'square_feet', 'year_built',
       'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'set', 'timestamp_2'],
      dtype='object')

__Mergin the test with the weather test data:__

In [14]:

df_test  = pd.merge(df_test,df_weather_test,on=['site_id','timestamp'],how='left')

__Create features from timestamp to represent (hour,day of the week,day,month) and also tranform the float time of some columns to reduce space:__

In [15]:
#(float types to reduce the space) https://www.kaggle.com/kulkarnivishwanath/ashrae-great-energy-predictor-iii-eda-model 
for df in [df_train, df_test]:
    df["month"] = df["timestamp_2"].dt.month.astype("uint8")
    df["day"] = df["timestamp_2"].dt.day.astype("uint8")
    df["day_of_week"] = df["timestamp_2"].dt.weekday.astype("uint8")
    df["hour"] = df["timestamp_2"].dt.hour.astype("uint8")
    df['weekend'] = np.where((df['day'] == 5) | (df['day'] == 6), 1, 0)
    df['air_temperature'] = df['air_temperature'].astype('float16')
    df['cloud_coverage'] = df['cloud_coverage'].astype("float16")
    df['dew_temperature'] = df['dew_temperature'].astype('float16')
    df['precip_depth_1_hr'] = df['precip_depth_1_hr'].astype('float32')
    df['sea_level_pressure'] = df['sea_level_pressure'].astype('float32')
    df['wind_direction'] = df['wind_direction'].astype('float32')
    df['wind_speed'] = df['wind_speed'].astype('float16')
    df['square_feet'] = np.log1p(df['square_feet']).astype("float32")

__Dropping not (anymore) usefull columns__

__NOTE!: 'floor_count','precip_depth_1_hr' are dropped because of the many NaN values!__


In [16]:

df_f_train=df_train.drop(['Unnamed: 0', 'timestamp','set','floor_count','precip_depth_1_hr',"timestamp_2"], axis=1)
df_f_test=df_test.drop(['timestamp','row_id','floor_count','precip_depth_1_hr',"timestamp_2"], axis=1)

In [18]:
df_f_train.head(5)

,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0.0,0,8.913685,2008.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
1,0,1,0,0.0,0,7.908755,2004.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
2,0,2,0,0.0,0,8.589886,1991.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
3,0,3,0,0.0,0,10.072639,2002.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
4,0,4,0,0.0,0,11.666574,1975.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0


In [19]:
df_f_test.head(5)

,building_id,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0,8.913685,2008.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
1,1,0,0,0,7.908755,2004.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
2,2,0,0,0,8.589886,1991.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
3,3,0,0,0,10.072639,2002.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
4,4,0,0,0,11.666574,1975.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0


### Dealing with NaN values

__Option 1:__ just drop the column! (hehe)

In [65]:
#df_f_train=df_f_train.dropna(axis=1, how='any', thresh=None, subset=None, inplace=False)
#df_f_test=df_f_test.dropna(axis=1, how='any', thresh=None, subset=None, inplace=False)

#Update:I found some problems here with the test data.

__Option 2:__ replace with the mean!
it will take some time to run this!

In [17]:
#Option 2: replace with the mean!
#it will take some time to run this!
print('starting to trainset')
df_f_train=df_f_train.fillna(df_f_train.mean(), inplace=False)
print('moving to testset')
df_f_test=df_f_test.fillna(df_f_test.mean(), inplace=False)
print('Done!')

starting to trainset
moving to testset
Done!


In [27]:
df_f_test.head(5)

,building_id,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0,8.913685,2008.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
1,1,0,0,0,7.908755,2004.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
2,2,0,0,0,8.589886,1991.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
3,3,0,0,0,10.072639,2002.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
4,4,0,0,0,11.666574,1975.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0


In [28]:
df_f_train.head(5)

,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0.0,0,8.913685,2008.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
1,0,1,0,0.0,0,7.908755,2004.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
2,0,2,0,0.0,0,8.589886,1991.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
3,0,3,0,0.0,0,10.072639,2002.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
4,0,4,0,0.0,0,11.666574,1975.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0


Next Step Ideas:

-Smarter ways to deal with NaN values.

-Dealing with missign values.

-Dealing with outliers.

-Standarize the meter readings?

### Reduce memory function (taken from kaggle)

In [ ]:
#source: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

In [18]:
import Reduce_memory as hey

In [19]:
df_f_train,Nan_list=hey.reduce_mem_usage(df_f_train)


Memory usage of properties dataframe is : 1484.52730178833  MB
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  meter_reading
dtype before:  float64
min for this col:  0.0
max for this col:  21904700.0
dtype after:  float32
******************************
******************************
Column:  primary_use
dtype before:  int8
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  float32
min for this col:  5

# Feature Creation

## ----On Construction----

Ideas:

-Create features that include the seasonal patters of each building.

-Include the results from clustering.

## ----On Construction----

Zo, we have df_f_train and df_f_test !!!!

# Modeling

### Testing modeling with basic train test split 

In [29]:
df_f_train.head(3)

,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0.0,0,8.913685,2008.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
1,0,1,0,0.0,0,7.908755,2004.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0
2,0,2,0,0.0,0,8.589886,1991.0,25.0,6.0,20.0,1019.700012,0.0,0.0,1,1,4,0,0


In [30]:
df_f_test.head(3)

,building_id,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0,8.913685,2008.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
1,1,0,0,0,7.908755,2004.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
2,2,0,0,0,8.589886,1991.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0


In [20]:
# free a bit the space

del df_train



### Model trainning function

In [21]:
def train_and_predict(train_x, train_y, val_x, model_cls, **model_params):
    model = model_cls(**model_params)
    model.fit(train_x, train_y)
    predictions = model.predict(val_x)
    fitted_model=model
    return predictions,fitted_model

# Cross validation

In [ ]:
def progress(text, verbose=True, same_line=False, newline_end=True):
    if verbose:
        print("{}[{}] {}".format("\r" if same_line else "", time.strftime("%Y-%m-%d %H:%M:%S"),
                                 text), end="\n" if newline_end else "")


def cross_validate(data, model_cls, ycol='MEDV', n_splits=5, **model_params):
    
    """Perform custom cross validation using randomly sequences of observations."""
    splitter = KFold(n_splits=n_splits, shuffle=False,random_state=3)

    scores = []
    for i, (train_index, val_index) in enumerate(splitter.split(data)):
        progress("Starting cross-validation fold {}.".format(i))

        # split the data according to the indices
        progress("Splitting data in train and validation sets.")
        train = data.iloc[train_index]
        val = data.iloc[val_index]

        # sample random sequences for training
        progress("Sampling {} sequences from training data.".format(len(train)))
        y_train = train[ycol]
        x_train = train.drop(ycol, axis=1)
        progress("Train set sampled.")

        # sample random sequences for validation
        progress("Sampling {} sequences from validation data.".format(len(val)))
        y_val = val[ycol]
        x_val = val.drop(ycol, axis=1)
        progress("Validation set sampled.")

        # train and predict validation set
        progress("Start training and predicting.")
        y_val_hat = train_and_predict(x_train, y_train, x_val, model_cls, **model_params)
        progress("Predictions on validation set made.")
        
        # evaluate using mean absolute error as an example
        #print(y_val.transpose())
        #print(y_val_hat[0])
        score = mean_absolute_error(y_val.values, y_val_hat[0])
        scores.append(score)
        progress("Validation score: {}.".format(score))

    return scores

In [ ]:
#import warnings
#warnings.filterwarnings('ignore')

#model_cls=LinearRegression
#model_params={'fit_intercept': True,'normalize': True }
#models_meters=[]
#for meteeer in [0,1,2,3]:
#    df_ff_train=df_f_train[df_f_train['meter']==meteeer]
#   
#
#    scores = cross_validate(df_ff_train[df_ff_train.columns.difference(['meter'])], model_cls, ycol='meter_reading', **model_params)
#    print("Mean validation score: {}".format(np.mean(scores)))
#    print('')
#    print('-------------------------------------------------------------------')
#    print('')
#    print('')
    

##### Example model run (that splits the data without cross validation)

In [22]:
def model_pred (data_x,data_y,model_cls,test_size =0.2,random_state=5,**model_params):
   
    X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, test_size = 0.1, random_state=5)

    train_x=X_train
    train_y=Y_train
    val_x=X_test

   # model_params={'fit_intercept': True }

    y_pred,fitted_model=train_and_predict(train_x, train_y, val_x, model_cls, **model_params)

    rmse = (np.sqrt(mean_squared_error(Y_test, y_pred)))

    print("The model performance for training set")
    print("--------------------------------------")
    print('RMSE is {}'.format(rmse))
    print("\n")
    return y_pred,fitted_model
    

### Use your model here with its model parameters:

In [63]:
#model_cls=XGBRegressor
#model_params= {
#    "n_estimators": 100,
#    "n_jobs": -1,
#    "verbose": 1
#}

model_cls=RandomForestRegressor

model_params= {
    "n_estimators": 15,
    "n_jobs": 6,
    "verbose": 1
}

#model_params= {
#    "n_estimators": 10,
#    "criterion": 'mae',
#    "n_jobs": -1,
#    "verbose": 1,
#}

# ######YOU CAN CHOOSE HERE FOR LINEAR REGRESSOR ALSO ##########

#model_cls=LinearRegression
#model_params={'fit_intercept': True,'normalize': True }
models_meters=[]
for meteeer in [0]:
    print("training meter ",meteeer)
    df_ff_train=df_f_train[df_f_train['meter']==meteeer]
    data_x=df_ff_train[df_ff_train.columns.difference(['meter_reading','meter'])]
    data_y=df_ff_train['meter_reading']

    y_pred,fitted_model=model_pred (data_x,data_y,model_cls,test_size =0.30,random_state=5,**model_params)
   #save the fitted model for each meter
    models_meters.append(fitted_model)
    
model_params= {
    "n_estimators": 15,
    "n_jobs": 2,
    "verbose": 1
}
for meteeer in [1,2,3]:
    print("training meter ",meteeer)
    df_ff_train=df_f_train[df_f_train['meter']==meteeer]
    data_x=df_ff_train[df_ff_train.columns.difference(['meter_reading','meter'])]
    data_y=df_ff_train['meter_reading']

    y_pred,fitted_model=model_pred (data_x,data_y,model_cls,test_size =0.10,random_state=7,**model_params)
    #save the fitted model for each meter
    models_meters.append(fitted_model)

training meter  0


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  15 out of  15 | elapsed:  9.7min finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  15 out of  15 | elapsed:    4.9s finished


The model performance for training set
--------------------------------------
RMSE is 86.48897769570057


training meter  1


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  5.3min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:    2.8s finished


The model performance for training set
--------------------------------------
RMSE is 1510.3691716359315


training meter  2


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  3.5min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:    1.7s finished


The model performance for training set
--------------------------------------
RMSE is 56292.35430993872


training meter  3


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  1.3min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


The model performance for training set
--------------------------------------
RMSE is 1201.541712408226




[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:    0.5s finished


In [42]:
df_f_train[(df_f_train['meter']==4) & (df_f_train['building_id']==0)]['meter_reading'].empty

True

### Run  model for each meter in the test data

In [56]:
df_test_w_index=df_f_test['meter'].reset_index()
    #for meteeer in [0,1,2,3]:

    
predictions = models_meters[0].predict(   df_f_test[df_f_test[df_f_test['meter']==0].columns.difference(['meter']) ]   )
df_row0=pd.DataFrame(df_test_w_index[df_test_w_index['meter']==0]['index']).join( pd.DataFrame(predictions) )
   
predictions = models_meters[1].predict(   df_f_test[df_f_test[df_f_test['meter']==1].columns.difference(['meter']) ]   )    
df_row1=pd.DataFrame(df_test_w_index[df_test_w_index['meter']==2]['index']).join( pd.DataFrame(predictions) )       


predictions = models_meters[2].predict(   df_f_test[df_f_test[df_f_test['meter']==2].columns.difference(['meter']) ]   )
df_row2=pd.DataFrame(df_test_w_index[df_test_w_index['meter']==1]['index']).join( pd.DataFrame(predictions) )    

    
predictions = models_meters[3].predict(   df_f_test[df_f_test[df_f_test['meter']==3].columns.difference(['meter']) ]   )
df_row3=pd.DataFrame(df_test_w_index[df_test_w_index['meter']==3]['index']).join( pd.DataFrame(predictions) )    


df_concat = pd.concat([df_row0, df_row1,df_row2,df_row3], axis=0)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  3.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  1.4min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:  1.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:   46.2s finished


In [57]:
building_models[0]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True),
 'nope',
 'nope',
 'nope']

In [59]:
df_f_test.head(10)

,building_id,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,0,8.913685,2008.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
1,1,0,0,0,7.908755,2004.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
2,2,0,0,0,8.589886,1991.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
3,3,0,0,0,10.072639,2002.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
4,4,0,0,0,11.666574,1975.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
5,5,0,0,0,8.987322,2000.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
6,6,0,0,4,10.237350,1981.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
7,7,0,0,0,11.704165,1989.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
8,7,1,0,0,11.704165,1989.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0
9,8,0,0,0,11.015510,2003.0,17.796875,4.0,11.703125,1021.400024,100.0,3.599609,1,1,6,0,0


In [57]:
df_concat.sort_values('index')

,index,0
0,0,0.000000
1,1,5.378580
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.307150
6,6,0.153575
7,7,0.000000
8,8,1670.432445
9,9,0.000000


In [58]:
df_test.head(5)

,row_id,building_id,meter,timestamp,timestamp_2,site_id,primary_use,square_feet,year_built,floor_count,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,day,day_of_week,hour,weekend
0,0,0,0,2017-01-01 00:00:00,2017-01-01,0,0,8.913685,2008.0,NaN,...,11.703125,NaN,1021.400024,100.0,3.599609,1,1,6,0,0
1,1,1,0,2017-01-01 00:00:00,2017-01-01,0,0,7.908755,2004.0,NaN,...,11.703125,NaN,1021.400024,100.0,3.599609,1,1,6,0,0
2,2,2,0,2017-01-01 00:00:00,2017-01-01,0,0,8.589886,1991.0,NaN,...,11.703125,NaN,1021.400024,100.0,3.599609,1,1,6,0,0
3,3,3,0,2017-01-01 00:00:00,2017-01-01,0,0,10.072639,2002.0,NaN,...,11.703125,NaN,1021.400024,100.0,3.599609,1,1,6,0,0
4,4,4,0,2017-01-01 00:00:00,2017-01-01,0,0,11.666574,1975.0,NaN,...,11.703125,NaN,1021.400024,100.0,3.599609,1,1,6,0,0


In [59]:
sample_submission.head(5)

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


### Save everything and create file for submiting your score to Kaggle!

In [60]:
submission=sample_submission.join( df_concat.sort_values('index') )[['row_id',0]] 
#submission


In [61]:
submission = submission.rename(columns={0: 'meter_reading'})

In [62]:
submission.to_csv('your4.csv', index=False, header = True)

In [63]:
submission.shape

(41697600, 2)

In [64]:
sample_submission.shape

(41697600, 2)

__Leaderboard score: 2.36  (place 1563 /1712  at time of submitting).__ (not that good!)

#### I believe we need standarization immediately! to improve!

# Next step ideas

__Data Prep:__

-Smarter ways to deal with NaN values.

-Dealing with missign values.

-Dealing with outliers.

-Standarize the meter readings?

__Feature Eng:__

-Create features that include the seasonal patters of each building.

-Include the results from clustering.



__Modeling:__
    
-Essemble models
-Grid search